# 2.2 数据操作

在深度学习中，我们通常会频繁地对数据进行操作。作为动手学深度学习的基础，本节将介绍如何对内存中的数据进行操作。

在PyTorch中，`Tensor`是一个类，也是存储和变换数据的主要工具。为了简洁，本书常将`Tensor`实例直接称作`Tensor`。如果你之前用过NumPy，你会发现`Tensor`和NumPy的多维数组非常类似。然而，`Tensor`提供GPU计算和自动求梯度等更多功能，这些使`Tensor`更加适合深度学习。

## 2.2.1 创建`Tensor`

我们先介绍`Tensor`的最基本功能。如果对这里用到的数学操作不是很熟悉，可以参阅附录中[“数学基础”](../chapter11_appendix/11.02_math.ipynb)一节。

首先从PyTorch导入`torch`模块。

In [1]:
import torch

torch.manual_seed(0) # 生成随机数用的种子
torch.cuda.manual_seed(0)

>㊟ [`torch.manual_seed()`](https://pytorch.org/docs/stable/torch.html#torch._C.Generator.manual_seed)的作用是设置用于生成随机数的种子。

>㊟ [`torch.cuda.manual_seed()`](https://pytorch.org/docs/stable/cuda.html#torch.cuda.manual_seed)作用与`torch.manual_seed()`相同，只不过是在当前GPU上执行，如果不可执行，比如根本就没有GPU，那么它也不报错。

然后我们用`torch.arange`函数创建一个行向量。

In [2]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

这时返回了一个`tensor`实例，其中包含了从0开始的12个连续整数。从打印`x`时显示的结果可以看出，这个`tensor`是长度为12的一维数组。

我们可以通过`shape`属性来获取`tensor`实例的形状。

In [3]:
x.shape

torch.Size([12])

下面使用`torch.reshape`函数把行向量`x`的形状改为(3, 4)，也就是一个3行4列的矩阵，并记作`X`。除了形状改变之外，`X`中的元素保持不变。

In [4]:
X = torch.reshape(x, (3, 4))
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

> ㊟ 常用改变tensor形状的函数是[`torch.Tensor.view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view),用法如下:

In [5]:
X2 = x.view(3, 4)
X2

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

注意`X`属性中的形状发生了变化。上面`x.reshape((3, 4))`也可写成`x.reshape((-1, 4))`或`x.reshape((3, -1))`。由于`x`的元素个数是已知的，这里的`-1`是能够通过元素个数和其他维度的大小推断出来的。

接下来，我们创建一个各元素为0，形状为(2, 3, 4)的张量。实际上，之前创建的向量和矩阵都是特殊的张量。

In [6]:
torch.zeros(2, 3, 4)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

类似地，我们可以创建各元素为1的张量。

In [7]:
torch.ones(3, 4)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

我们也可以通过Python的列表（list）指定需要创建的`NDArray`中每个元素的值。

In [8]:
Y = torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
Y

tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])

有些情况下，我们需要随机生成`Tensor`中每个元素的值。下面我们创建一个形状为(3, 4)的`Tensor`。它的每个元素都随机采样于均值为0、标准差为1的正态分布。

In [9]:
torch.normal(0, 1, size=(3, 4))

tensor([[ 1.5410, -0.2934, -2.1788,  0.5684],
        [-1.0845, -1.3986,  0.4033,  0.8380],
        [-0.7193, -0.4033, -0.5966,  0.1820]])

> ㊟ [`torch.normal()`](https://pytorch.org/docs/stable/torch.html#torch.normal)和[`torch.randn()`](https://pytorch.org/docs/stable/torch.html#torch.randn)都可以返回一个正态分布的随机数组成的`tensor`。`torch.randn()`的功能就是返回一个标准的正态分布(均值为$0$,标准差为$1$)随机数`tensor`,用法如下例:☟

In [16]:
torch.randn(4, 4)

tensor([[ 1.5091,  2.0820,  1.7067,  2.3804],
        [-1.1256, -0.3170, -1.0925, -0.0852],
        [ 0.3276, -0.7607, -1.5991,  0.0185],
        [-0.7504,  0.1854,  0.6211,  0.6382]])

## 2.2.2 运算

`torch.Tensor`支持大量的运算符（operator）。例如，我们可以对之前创建的两个形状为(3, 4)的`Tensor`做按元素加法。所得结果形状不变。

In [17]:
X + Y

tensor([[ 2,  2,  6,  6],
        [ 5,  7,  9, 11],
        [12, 12, 12, 12]])

按元素乘法：

In [18]:
X * Y

tensor([[ 0,  1,  8,  9],
        [ 4, 10, 18, 28],
        [32, 27, 20, 11]])

按元素除法：

In [19]:
X / Y

tensor([[ 0,  1,  0,  1],
        [ 4,  2,  2,  1],
        [ 2,  3,  5, 11]])

按元素做指数运算：

In [20]:
Y.to(dtype=torch.float64).exp() #得先把Y转换成浮点

tensor([[ 7.3891,  2.7183, 54.5982, 20.0855],
        [ 2.7183,  7.3891, 20.0855, 54.5982],
        [54.5982, 20.0855,  7.3891,  2.7183]], dtype=torch.float64)

除了按元素计算外，我们还可以使用[`torch.mm()`](https://pytorch.org/docs/stable/torch.html#torch.mm)函数做矩阵乘法。下面将`X`与`Y`的转置做矩阵乘法。由于`X`是3行4列的矩阵，`Y`转置为4行3列的矩阵，因此两个矩阵相乘得到3行3列的矩阵。

In [22]:
print(X)
print(Y)
print(torch.mm(X, Y.T))

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])
tensor([[ 18,  20,  10],
        [ 58,  60,  50],
        [ 98, 100,  90]])


我们也可以将多个`Tensor`连结（concatenate）。下面分别在行上（维度0，即形状中的最左边元素）和列上（维度1，即形状中左起第二个元素）连结两个矩阵。可以看到，输出的第一个`Tensor`在维度0的长度（$6$）为两个输入矩阵在维度0的长度之和（$3+3$），而输出的第二个`Tensor`在维度1的长度（$8$）为两个输入矩阵在维度1的长度之和（$4+4$）。

In [16]:
print( torch.cat((X, Y),0) )
print( torch.cat((X, Y),1) )

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [ 2,  1,  4,  3],
        [ 1,  2,  3,  4],
        [ 4,  3,  2,  1]])
tensor([[ 0,  1,  2,  3,  2,  1,  4,  3],
        [ 4,  5,  6,  7,  1,  2,  3,  4],
        [ 8,  9, 10, 11,  4,  3,  2,  1]])


使用条件判断式可以得到元素为0或1的新的`Tensor`。以`X == Y`为例，如果`X`和`Y`在相同位置的条件判断为真（值相等），那么新的`Tensor`在相同位置的值为1；反之为0。

In [23]:
(X == Y).to(torch.float64)

tensor([[0., 1., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=torch.float64)

对`Tensor`中的所有元素求和得到只有一个元素的`Tensor`。

In [24]:
X.sum()

tensor(66)

我们可以通过[`item()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.item)函数将结果变换为Python中的标量。下面例子中`X`的$L_2$范数结果同上例一样是单元素`Tensor`，但最后结果变换成了Python中的标量。

In [19]:
X.to(torch.float64).norm().item()

22.494443758403985

> ㊟ [`torch.norm()`](https://pytorch.org/docs/stable/torch.html#torch.norm)函数的作用是求`tensor`的范数。

我们也可以把`Y.to(torch.float64).exp()`、`X.sum()`、`X.to(torch.float64).norm()`等分别改写为`torch.exp(Y.to(torch.float64))`、`torch.sum(X)`、`torch.norm(X.to(torch.float64))`等。

## 2.2.3 广播机制

前面我们看到如何对两个形状相同的`Tensor`做按元素运算。当对两个形状不同的`Tensor`按元素运算时，可能会触发广播（broadcasting）机制：先适当复制元素使这两个`Tensor`形状相同后再按元素运算。

定义两个`Tensor`：

In [25]:
A = torch.arange(3).reshape((3, 1))
B = torch.arange(2).reshape((1, 2))
A, B

(tensor([[0],
         [1],
         [2]]), tensor([[0, 1]]))

由于`A`和`B`分别是3行1列和1行2列的矩阵，如果要计算`A + B`，那么`A`中第一列的3个元素被广播（复制）到了第二列，而`B`中第一行的2个元素被广播（复制）到了第二行和第三行。如此，就可以对2个3行2列的矩阵按元素相加。

In [26]:
A + B

tensor([[0, 1],
        [1, 2],
        [2, 3]])

## 2.2.4 索引

在`Tensor`中，索引（index）代表了元素的位置。`Tensor`的索引从0开始逐一递增。例如，一个3行2列的矩阵的行索引分别为0、1和2，列索引分别为0和1。

在下面的例子中，我们指定了`Tensor`的行索引截取范围`[1:3]`。依据左闭右开指定范围的惯例，它截取了矩阵`X`中行索引为1和2的两行。

In [29]:
print(X)
X[1:3]

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


tensor([[ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

我们可以指定`Tensor`中需要访问的单个元素的位置，如矩阵中行和列的索引，并为该元素重新赋值。

In [31]:
X[1, 2] = 9
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  9,  7],
        [ 8,  9, 10, 11]])

当然，我们也可以截取一部分元素，并为它们重新赋值。在下面的例子中，我们为行索引为1的每一列元素重新赋值。

In [24]:
X[1:2, :] = 12
X

tensor([[ 0,  1,  2,  3],
        [12, 12, 12, 12],
        [ 8,  9, 10, 11]])

## 2.2.5 运算的内存开销

在前面的例子里我们对每个操作新开内存来存储运算结果。举个例子，即使像`Y = X + Y`这样的运算，我们也会新开内存，然后将`Y`指向新内存。为了演示这一点，我们可以使用Python自带的`id`函数：如果两个实例的ID一致，那么它们所对应的内存地址相同；反之则不同。

In [25]:
before = id(Y)
Y = Y + X
id(Y) == before

False

如果想指定结果到特定内存，我们可以使用前面介绍的索引来进行替换操作。在下面的例子中，我们先通过`zeros_like`创建和`Y`形状相同且元素为0的`Tensor`，记为`Z`。接下来，我们把`X + Y`的结果通过`[:]`写进`Z`对应的内存中。

In [32]:
Z = torch.zeros(Y.shape)
before = id(Z)
Z[:] = X + Y
id(Z) == before

True

实际上，上例中我们还是为`X + Y`开了临时内存来存储计算结果，再复制到`Z`对应的内存。如果想避免这个临时内存开销，我们可以使用运算符全名函数中的`out`参数。

In [27]:
torch.add(X, Y, out=Z)
id(Z) == before

True

如果`X`的值在之后的程序中不会复用，我们也可以用 `X[:] = X + Y` 或者 `X += Y` 来减少运算的内存开销。

In [59]:
before = id(X)
X += Y
id(X) == before

True

## 2.2.6 `Tensor`和NumPy相互变换

我们可以通过[`tensor()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor)函数和[`numpy()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.numpy)函数令数据在`Tensor`和NumPy格式之间相互变换。下面将NumPy实例变换成`Tensor`实例。

In [33]:
import numpy as np

P = np.ones((2, 3))
D = torch.tensor(P)
D

tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)

再将`Tensor`实例变换成NumPy实例。

In [63]:
D.numpy()

array([[1., 1., 1.],
       [1., 1., 1.]])

## 小结

* `Tensor`是PyTorch中存储和变换数据的主要工具。
* 可以轻松地对`Tensor`创建、运算、指定索引，并与NumPy之间相互变换。

## 练习

* 运行本节中的代码。将本节中条件判断式`X == Y`改为`X < Y`或`X > Y`，看看能够得到什么样的`Tensor`。
* 将广播机制中按元素运算的两个`Tensor`替换成其他形状，结果是否和预期一样？

## 扫码直达[知乎专栏](https://zhuanlan.zhihu.com/unicom-d2l)

![](../img/zhihu.png)